In [1]:
import numpy
import numpy as np
import pandas
import pandas as pd
from numba import njit
import sys,glob
import zipfile, os, time
import urllib3
from datetime import datetime, timedelta
import glob
import h5py
sys.path.append(os.getcwd()+'/../cds-backend/code/')
sys.path.append(os.getcwd()+'/../harvest/code/')
from harvest_convert_to_netCDF_newfixes import write_dict_h5
import cds_eua3 as eua
eua.logging_set_level(30)
import xarray as xr

import cdsapi, zipfile, os, time
#import schedule
import copy
from shutil import copyfile
import multiprocessing
import pickle

import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.pylab as pylab


In [ ]:
# t0 = time.time()

# c = cdsapi.Client()
# r = c.retrieve(
#     'insitu-comprehensive-upper-air-observation-network',
#     {
#         'variable':['air_temperature',],
#         'statid':'all',
#         'date': '19780101-20181231',
#         'pressure_level': 100,
#     })
# if True:
#     # Start Download
#     r.download(target='download.zip')
#     # Check file size
#     assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
    
# print("Time elapsed: ", time.time()-t0, "s")
# z = zipfile.ZipFile('download.zip')
# print("Unzipping retrieved files")
# z.extractall(path='./comp100hpa')
# z.close()
# os.remove('download.zip')


In [ ]:
compf = glob.glob('./comp100hpa/*.nc')
print(len(compf))

In [ ]:
ds = xr.open_dataset(compf[0])
df = ds.to_dataframe()
np.asarray(df.ta)

In [ ]:
compf = glob.glob('./comp100hpa/*.nc')[:100]


In [ ]:
ds = xr.open_dataset(compf[1])
df = ds.to_dataframe()

In [ ]:
tt=time.time()
ds = xr.open_dataset(compf[1])
a = np.array(ds.time.values)
a.astype('M8[D]')
# np.unique(a)

In [ ]:
ds.time.values

In [ ]:
ds = xr.open_dataset(compf[1])
ds = ds.where(ds.time < np.datetime64('2006-12-31T00:23:59.999999999'), drop=True)
ds.time.values

In [17]:
stations = pickle.load( open( "stations.p", "rb" ))
out = stations[0:0]

start = 1978
end = 2006
dtend = np.datetime64('2006-12-31T00:23:59.999999999')
intervall = end - start
compf = glob.glob('./comp100hpa/*.nc')[:5]
for i in compf:
    with xr.open_dataset(i) as ds:
        ds = ds.where(ds.time < dtend, drop=True)
        timelen = len(np.unique(ds.time.values.astype('M8[D]')))
        print(timelen)
    #     if timelen >= 0.9*intervall*365:
    #         selected = np.asarray(ds.ta.values)
    #         datlen = len(selected)
    #         try:
    #             coefficients, residuals, _, _, _ = np.polyfit(range(datlen),selected,1,full=True)
    #             trend = coefficients[0]*(10/intervall*len(ds.ta.values))
    #             out.loc[len(out.index)] = [trend, ds.lon.values[0], ds.lat.values[0]]
    #         except:
    #             pass
# pickle.dump( out, open( "COMP_100hPa_1978_2006_Trend.p", "wb" ))

153
5194
0
6221
632


In [ ]:
out

In [ ]:
stations = pickle.load( open( "stations.p", "rb" ))
start = 1978
end = 2006
intervall = end - start
mindata = int(intervall*0.9)
for i in range(len(stations.station_name)):
    files = glob.glob('/raid60/scratch/uli/IGRA_Data/temp300h/IGRA_H_' + stations.station_name.iloc[i] + '*.p')
#     files = glob.glob('/raid60/scratch/uli/IGRA_Data/temp300/IGRA_' + stations.station_name.iloc[i] + '*.p')
    filesinintervall = []
    for o in files:
        yr = int(o.split('_')[-2])
        if yr <= end and yr >= start:
            filesinintervall.append(o)
    if len(filesinintervall) < mindata:
        stations.station_name.iloc[i] = np.nan
    else:
        temp = []
        for j in filesinintervall:
            temp.extend(pickle.load( open( j, "rb" )))
        selected = np.asarray(temp)[~np.isnan(np.asarray(temp))]
        datlen = len(selected)
        try:
            coefficients, residuals, _, _, _ = np.polyfit(range(datlen),selected,1,full=True)
            stations.station_name.iloc[i] = coefficients[0]*(10/intervall*len(temp))
        except:
            stations.station_name.iloc[i] = np.nan
pickle.dump( stations, open( "IGRA_H_300hPa_1978_2006_Trend.p", "wb" ))
# pickle.dump( stations, open( "IGRA_300hPa_1978_2018_Trend.p", "wb" ))

In [10]:
files = glob.glob('/raid60/scratch/uli/IGRA_H/*/*global_cdm-lev.csv')
# print(len(files_h))
# files_h

In [3]:
# def savetoarray(file, dataset: str='IGRA_H', dest: str='./temp100/'):
#     yr = file.split('/')[5]
#     plev = 10000
#     chunk = pd.read_csv(file, header=12, index_col=False, usecols=['station_name', 'observed_value', 'air_pressure'])
#     stations = chunk.station_name.drop_duplicates()
#     for i in stations:
#         if not os.path.isfile( dest + dataset + '_' + i + '_' + yr + '_' + str(plev) + ".p"):
#             da = chunk[chunk.station_name == i]
#             da = da[da.air_pressure == plev]
#             pickle.dump( np.asarray(da.observed_value), open( dest + dataset + '_' + i + '_' + yr + '_' + str(plev) + ".p", "wb" ))

---

In [4]:
# stations = []
# for i in range(len(files)):
#     if i == 0:
#         chunk = pd.read_csv(files[i], header=12, index_col=False, usecols=['location_longitude', 'location_latitude', 'station_name'])
#         stations = chunk.drop_duplicates(['station_name'])
#     else:
#         chunk = pd.read_csv(files[i], header=12, index_col=False, usecols=['location_longitude', 'location_latitude', 'station_name'])
#         stations = pd.concat([stations, chunk.drop_duplicates(['station_name'])], ignore_index=True)
# stations = stations.drop_duplicates('station_name')
# pickle.dump( stations, open( "stations.p", "wb" ))

In [22]:
stations = pickle.load( open( "stations.p", "rb" ))
stations

,station_name,location_longitude,location_latitude
0,KRM00091487,-159.3670,3.8500
1,CHM00055664,87.0830,28.6330
2,CHM00056444,98.8830,28.4500
3,BMM00048062,92.8830,20.1330
4,CHM00057290,114.0500,32.9670
...,...,...,...
36592,INM00043110,73.3333,16.9833
37459,INM00042165,73.3000,28.0000
37460,INM00042328,70.9167,26.9000
37461,RPM00098233,121.7525,17.6375


## 1979-2019, sowie 1979-2006

In [3]:
stations = pickle.load( open( "stations.p", "rb" ))
start = 1978
end = 2018
intervall = end - start
mindata = int(intervall*0.9)
for i in range(len(stations.station_name)):
    files = glob.glob('/raid60/scratch/uli/IGRA_Data/temp300/IGRA_' + stations.station_name.iloc[i] + '*.p')
#     files = glob.glob('/raid60/scratch/uli/IGRA_Data/temp300/IGRA_' + stations.station_name.iloc[i] + '*.p')
    filesinintervall = []
    for o in files:
        yr = int(o.split('_')[-2])
        if yr <= end and yr >= start:
            filesinintervall.append(o)
    if len(filesinintervall) < mindata:
        stations.station_name.iloc[i] = np.nan
    else:
        temp = []
        for j in filesinintervall:
            temp.extend(pickle.load( open( j, "rb" )))
        selected = np.asarray(temp)[~np.isnan(np.asarray(temp))]
        datlen = len(selected)
        try:
            coefficients, residuals, _, _, _ = np.polyfit(range(datlen),selected,1,full=True)
            stations.station_name.iloc[i] = coefficients[0]*(10/intervall*len(temp))
        except:
            stations.station_name.iloc[i] = np.nan
pickle.dump( stations, open( "IGRA_300hPa_1978_2018_Trend.p", "wb" ))
# pickle.dump( stations, open( "IGRA_300hPa_1978_2018_Trend.p", "wb" ))

In [4]:
trends = glob.glob('IGRA*Trend.p')
trends

['IGRA_300hPa_1978_2018_Trend.p',
 'IGRA_100hPa_1978_2018_Trend.p',
 'IGRA_H_300hPa_1978_2018_Trend.p',
 'IGRA_H_300hPa_1978_2006_Trend.p',
 'IGRA_300hPa_1978_2006_Trend.p',
 'IGRA_100hPa_1978_2006_Trend.p',
 'IGRA_H_50hPa_1978_2018_Trend.p',
 'IGRA_50hPa_1978_2018_Trend.p',
 'IGRA_50hPa_1978_2006_Trend.p',
 'IGRA_H_50hPa_1978_2006_Trend.p',
 'IGRA_H_100hPa_1978_2006_Trend.p',
 'IGRA_H_100hPa_1978_2018_Trend.p']

In [5]:
def plt_trends(i):
    params = {'legend.fontsize': 'x-large',
              'figure.figsize': (12, 8),
             'axes.labelsize': 'x-large',
             'axes.titlesize': 20,
             'xtick.labelsize':'medium',
             'ytick.labelsize':'medium'}
    pylab.rcParams.update(params)

    igra = pickle.load( open( i, "rb" ))

    ax = plt.axes([0, 0, 1, 1], projection=ccrs.PlateCarree())
    ax.add_feature(cfeature.OCEAN, zorder=0)
    ax.coastlines()

    da = igra.dropna()
    reduced = np.asarray(da['station_name'])
    longitudes = list(da['location_longitude'])
    latitudes = list(da['location_latitude'])
    plt.scatter(longitudes, latitudes, s=40, alpha=1,
                c= da['station_name'],
                cmap='seismic',
                vmin=-2,
                vmax=2)
    plt.colorbar(orientation='horizontal', label='K/10a', shrink=0.9, pad=0.05)
    plt.tight_layout()
    plt.title(i.split('.')[0])
    plt.savefig('plots/'+i+'ng', bbox_inches='tight')
    plt.close()


In [6]:
for i in trends:
    plt_trends(i)

<ipython-input-5-d1049efc05a0>:26: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
<ipython-input-5-d1049efc05a0>:26: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
<ipython-input-5-d1049efc05a0>:26: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
<ipython-input-5-d1049efc05a0>:26: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
<ipython-input-5-d1049efc05a0>:26: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
<ipython-input-5-d1049efc05a0>:26: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.t

---

---

In [11]:
da['station_name']*3650

59      -319.228
67      -759.201
89      -555.898
91      -81.1182
99      -182.975
          ...   
4822     179.013
4835     3765.76
4838      218.52
4839     28.4268
5625   -0.268902
Name: station_name, Length: 394, dtype: object

In [12]:
reduced.min()

-1.0927977002580942

In [13]:
import matplotlib.cm as cm
# colors = np.array(colors, dtype=np.float)
# col = [cm.tab20(x) for x in range(len(reduced))]
# col = [cm.tab20(x/len(reduced)) for x in range(len(reduced))]


In [14]:
col

NameError: name 'col' is not defined

In [ ]:
def savetoarray(file, dataset: str='IGRA', dest: str='./temp100/'):
    yr = file.split('/')[5]
    plev = 10000
    chunk = pd.read_csv(file, header=12, index_col=False, usecols=['station_name', 'observed_value', 'air_pressure'])
    stations = chunk.station_name.drop_duplicates()
    for i in stations:
        if not os.path.isfile( dest + dataset + '_' + i + '_' + yr + '_' + str(plev) + ".p"):
            da = chunk[chunk.station_name == i]
            da = da[da.air_pressure == plev]
            pickle.dump( np.asarray(da.observed_value), open( dest + dataset + '_' + i + '_' + yr + '_' + str(plev) + ".p", "wb" ))

In [ ]:
if __name__ == '__main__':
    pool = multiprocessing.Pool(processes=20)
    result_list = pool.map(savetoarray, files[10:20])
    print(result_list)

In [ ]:
print('!')

In [ ]:
lon = igra.report_timestamp.drop_duplicates()
lat = igra.location_longitude.drop_duplicates()
print(len(lon))
print(len(lat))

In [ ]:
# IGRA and its harmonized version
for i in range(len(files)):
    print(i)
    if i == 0:
#         igra = pandas.read_csv(files[i], header=12)
        igrah = pandas.read_csv(files_h[i], header=12)
    else:
#         igra = igra.append(pandas.read_csv(files[i], header=12))
        igrah = igrah.append(pandas.read_csv(files_h[i], header=12))

In [ ]:
preslist = [10000, 20000, 50000, 85000,]
for i in range(len(preslist)):
    c = comp[comp.plev == preslist[i]]
    c.time = pandas.to_datetime(c.time, utc=True)
    xax = np.asarray(c.time)
    
    selected = c.ta
    datlen = len(selected)
    coefficients, residuals, _, _, _ = np.polyfit(range(datlen),selected,1,full=True)
    mse = residuals[0]/(datlen)
    nrmse = np.sqrt(mse)/(selected.max() - selected.min())

    fig, (ax1) = plt.subplots(1, 1)
    ax1.plot(xax, np.asarray(selected), label = 'COMP', alpha=0.7)
    ax1.plot(xax, [coefficients[0]*x + coefficients[1] for x in range(len(selected))], label = 'COMP trend')
    ax1.set(xlabel ='time [multiples of 12h]', ylabel='temperature [K]', title='COMP Trend '+str(int(preslist[i]/100))+'hPa '+loc)
    textstr = 'Slope ' + str(coefficients[0]) +'\n'+'NRMSE: ' + str(nrmse)
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)
    ax1.text(0.03, 0.95, textstr, transform=ax1.transAxes, fontsize=16, verticalalignment='top', bbox=props)


    ax1.legend( loc='best')
    plt.show()
    plt.close(fig)